Импорт библиотек

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import telebot
from telebot import types

Подключение к таблице для создания датафрейма

In [2]:
df_shedule = pd.read_excel('..\SheduleFESU.xlsx', header=0, usecols='A:J')
# df_shedule.loc[df_shedule["Profile"] == ' Социология', "Profile"] = "Социология" #df = df.replace(to_replace=r'.*', value=new_value, regex=True)
df_shedule.fillna(value='Окно', inplace=True)
display(df_shedule.isna().sum())


Courses       0
Profile       0
Group         0
Num_Day       0
Day           0
Num_Lesson    0
Subject       0
Room          0
Week          0
Logo          0
dtype: int64

In [3]:
group_names = df_shedule['Group'].unique()
len(group_names)
logo = "https://img.icons8.com/?size=100&id=EVWPoM8ohVl2&format=png&color=000000"

nm_lesson = df_shedule.loc[df_shedule['Num_Day'] == 2, 'Num_Lesson'].values
nm_lesson
subject = df_shedule.loc[(df_shedule['Num_Day'] == int(1)) & (df_shedule['Group'] == int(9409)), 'Subject'].values
subject

array(['Иностранный язык ', 'Культурология', 'Основы социологии', 'ИКТ'],
      dtype=object)

In [4]:
name_days = pd.DataFrame({
        "1": ["Понедельник"],
        "2": ["Вториник"],
        "3": ["Среда"],
        "4": ["Четверг"],
        "5": ["Пятница"],
        "6": ["Суббота"],
        "7": ["Воскресенье"]}) #, index=range(7))

name_days = name_days.stack().reset_index(drop=True)
name_days

0    Понедельник
1       Вториник
2          Среда
3        Четверг
4        Пятница
5        Суббота
6    Воскресенье
dtype: object

Проверка логики программы

In [5]:
def get_text_messages( message_group = input('Enter num group: '), message_day = input('Enter num day: ') ): #message_course = input('Enter num course: '),
    num_of_week_day = message_day.strip()
    # num_of_course = message_course.strip()
    num_of_group = message_group.strip()
    subject = df_shedule.loc[(df_shedule['Num_Day'] == int(num_of_week_day)) & (df_shedule['Group'] == int(num_of_group)), 'Subject'].values
    nm_lesson = df_shedule.loc[(df_shedule['Num_Day'] == int(num_of_week_day)) & (df_shedule['Group'] == int(num_of_group)), 'Num_Lesson'].values
    # print(f'{num_of_course} {num_of_group} {num_of_week_day}')
    
    if len(subject) > 0:
        i = 0
        for sub in subject:
            print(f"{df_shedule[df_shedule['Num_Day'] == int(num_of_week_day)]['Day'].values[0]} Пара # {nm_lesson[i]} по дисциплине: {sub}")
            i += 1

    else: 
        print(f'В {[name_days[int(num_of_week_day)- 1]][0]} у Вас пар нет')


get_text_messages()

В Понедельник у Вас пар нет


# Create bot Telegram

In [6]:
# 1)Create bot 2)Give token 3)Task like Neyro 4) Add pandas 5)Thin abot Logic 1010)task server Heroku -> RSU
load_dotenv()
TOKEN_BOT_RSU = os.getenv('TOKEN_BOT_RSU') 


In [ ]:
bot = telebot.TeleBot(TOKEN_BOT_RSU)
@bot.message_handler(commands=['start'])  #, 'help'
def handle_start(message):
    mess = f'Привет!, <b> {message.from_user.first_name} <u>{message.from_user.last_name}</u> </b>'
    bot.send_message(message.chat.id, mess, parse_mode = 'html' )

# @bot.message_handler()  # All message or text
# def handle_start(message):
#     if message.text.upper().lower() == "Привет":
#         bot.send_message(message.chat.id, f"И тебе привет {message.from_user.first_name}", parse_mode = 'html' )
#     elif message.text == "id":
#         bot.send_message(message.chat.id, f"Твой ID {message.from_user.id}", parse_mode = 'html' )
#     elif message.text == 'img':
#         photo = open("image_fx_ (BI).jpg", 'rb')
#         bot.send_photo(message.chat.id, photo=photo)
#     else:
#         bot.send_message(message.chat.id, f"Извини - не понимаю", parse_mode = 'html' )


# @bot.message_handler(content_types = ['photo'])
# def get_user_photo(message):
#     bot.send_message(message.chat.id, "Вы прислали картинку, а я ожидаю текст")

# @bot.message_handler(commands=['website']) # кнопка с ссылкой на сайт
# def website(message): # Для кнопок
#     markup = types.InlineKeyboardMarkup()
#     markup.add(types.InlineKeyboardButton("Перейти на сайт", url="ya.ru"))
#     bot.send_message(message.chat.id, f"Твой ID {message.from_user.id} переходи", reply_markup=markup)


# Функция для обработки команды /help
@bot.message_handler(commands=['help'])
def welcome(message): # Для кнопок внутри меню
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=4, one_time_keyboard=True) # Добавление адаптивных кнопок
    First = types.KeyboardButton('Первый')
    Second = types.KeyboardButton('Второй')
    Third = types.KeyboardButton('Третий')
    Fourth = types.KeyboardButton('Четветрый')
    markup.add(First, Second, Third, Fourth)

    bot.send_message(message.chat.id, f"Привет {message.from_user.first_name}, выбери курс", reply_markup=markup)

# Обработка нажатий кнопок
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    if message.text == 'Первый':
        bot.reply_to(message, "Вы выбрали первый вариант.")
    elif message.text == 'Второй':
        bot.reply_to(message, "Вы выбрали второй вариант.")
    else:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=4, one_time_keyboard=True)
        Three = types.KeyboardButton('Третий2')
        Four = types.KeyboardButton('Четветрый2') 
        bot.reply_to(message, )


    # @bot.message_handler(content_types = ['text'])
    # def get_text_messages(message_group = input('Enter num group: '), message_day = input('Enter num day: ') ): #message_course = input('Enter num course: '),
    #     num_of_week_day = message_day.strip()
    #     # num_of_course = message_course.strip()
    #     num_of_group = message_group.strip()
    #     subject = df_shedule.loc[(df_shedule['Num_Day'] == int(num_of_week_day)) & (df_shedule['Group'] == int(num_of_group)), 'Subject'].values
    #     nm_lesson = df_shedule.loc[(df_shedule['Num_Day'] == int(num_of_week_day)) & (df_shedule['Group'] == int(num_of_group)), 'Num_Lesson'].values
    #     # print(f'{num_of_course} {num_of_group} {num_of_week_day}')
        
    #     if len(subject) > 0:
    #         i = 0
    #         for sub in subject:
    #             print(f"{df_shedule[df_shedule['Num_Day'] == int(num_of_week_day)]['Day'].values[0]} Пара # {nm_lesson[i]} по дисциплине: {sub}")
    #             i += 1

    #     else: 
    #         print(f'В {[name_days[int(num_of_week_day)- 1]][0]} у Вас пар нет')


    # get_text_messages()

bot.polling(none_stop = True, interval = 0)

variant two


In [ ]:
bot = telebot.TeleBot(TOKEN_BOT_RSU) # Задаем бот
# @bot.message_handler(commands=['start'])  #, 'help'
# def handle_start(message):
#     mess = f'Привет!, <b> {message.from_user.first_name} <u>{message.from_user.last_name}</u> </b>'
#     bot.send_message(message.chat.id, mess, parse_mode = 'html' )

@bot.message_handler(commands=['start'])
def welcome(message): # Для кнопок внутри меню
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=4, one_time_keyboard=True) # Добавление адаптивных кнопок
    First = types.KeyboardButton('Первый')
    Second = types.KeyboardButton('Второй')
    Third = types.KeyboardButton('Третий')
    Fourth = types.KeyboardButton('Четветрый')
    markup.add(First, Second, Third, Fourth)

    bot.send_message(message.chat.id, f"Привет {message.from_user.first_name}, выбери курс", reply_markup=markup)

# Обработка нажатий кнопок
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    if message.text == 'Первый':
        bot.reply_to(message, "Вы выбрали первый вариант.")
    elif message.text == 'Второй':
        bot.reply_to(message, "Вы выбрали второй вариант.")
    else:
        bot.reply_to(message, "В34")


# Первая группа вложенных кнопок
def first_level_choice(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True,row_width=2, one_time_keyboard=True)
    btn_suboption11 = types.KeyboardButton('Подвариант 1.1')
    btn_suboption12 = types.KeyboardButton('Подвариант 1.2')
    markup.add(btn_suboption11, btn_suboption12)
    
    bot.send_message(message.chat.id, "Вы выбрали Первый вариант. Теперь выберите подвариант:", reply_markup=markup)

bot.polling(none_stop = True, interval = 0)